In [ ]:
# Mount + installs
from google.colab import drive
drive.mount('/content/drive')

!pip -q install imbalanced-learn xgboost lightgbm shap

# Paths & basics
BASE = "/content/drive/MyDrive/heartriskx"
DATA = f"{BASE}/data"
OUT  = f"{BASE}/outputs/day8"
MODELS = f"{BASE}/models/day8"

import os, json, warnings
os.makedirs(OUT, exist_ok=True)
os.makedirs(MODELS, exist_ok=True)
warnings.filterwarnings("ignore")

RANDOM_STATE = 42

# Core imports
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, average_precision_score, brier_score_loss,
                             classification_report, confusion_matrix)
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.inspection import PartialDependenceDisplay

from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import shap, joblib

np.random.seed(RANDOM_STATE)


Mounted at /content/drive


In [ ]:
# --- Heart2020 ---
heart2020 = pd.read_csv(f"{DATA}/heart_2020.csv")
heart2020["target"] = heart2020["HeartDisease"].map({"Yes":1,"No":0}).astype(int)
heart2020 = heart2020.drop(columns=["HeartDisease"])

# --- Cardio ---
cardio = pd.read_csv(f"{DATA}/cardio_train.csv", sep=";")
cardio = cardio.rename(columns={"cardio":"target"})

# --- UCI (Cleveland) ---
uci = pd.read_csv(f"{DATA}/uci_heart.csv", header=None)
uci.columns = ["age","sex","cp","trestbps","chol","fbs","restecg",
               "thalach","exang","oldpeak","slope","ca","thal","target"]
uci["target"] = (uci["target"] > 0).astype(int)

print("Heart2020:", heart2020.shape, heart2020["target"].value_counts().to_dict())
print("Cardio   :", cardio.shape,    cardio["target"].value_counts().to_dict())
print("UCI      :", uci.shape,       uci["target"].value_counts().to_dict())

# Train/test splits
Xh, yh = heart2020.drop(columns=["target"]), heart2020["target"]
Xh_tr, Xh_te, yh_tr, yh_te = train_test_split(Xh, yh, test_size=0.2, stratify=yh, random_state=RANDOM_STATE)

Xc, yc = cardio.drop(columns=["target","id"]), cardio["target"]
Xc_tr, Xc_te, yc_tr, yc_te = train_test_split(Xc, yc, test_size=0.2, stratify=yc, random_state=RANDOM_STATE)

Xu, yu = uci.drop(columns=["target"]), uci["target"]
Xu_tr, Xu_te, yu_tr, yu_te = train_test_split(Xu, yu, test_size=0.2, stratify=yu, random_state=RANDOM_STATE)


Heart2020: (319795, 18) {0: 292422, 1: 27373}
Cardio   : (70000, 13) {0: 35021, 1: 34979}
UCI      : (303, 14) {0: 164, 1: 139}


In [ ]:
def make_preprocessor(X, scale_numeric=False):
    num_cols = X.select_dtypes(include=np.number).columns.tolist()
    cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
    transformers = []

    if num_cols:
        transformers.append(("num", StandardScaler() if scale_numeric else "passthrough", num_cols))
    if cat_cols:
        transformers.append(("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols))
    return ColumnTransformer(transformers)

def best_f1_threshold(y_true, proba):
    from sklearn.metrics import precision_recall_curve
    p, r, th = precision_recall_curve(y_true, proba)
    f1 = 2*p*r/(p+r+1e-9)
    i = int(np.nanargmax(f1))
    return float(th[max(0,i-1)] if i==len(th) else th[i]), float(f1[i]), float(p[i]), float(r[i])

def report_test_metrics(name, y_true, proba, thr=0.5):
    pred = (proba >= thr).astype(int)
    acc = accuracy_score(y_true, pred)
    prec= precision_score(y_true, pred, zero_division=0)
    rec = recall_score(y_true, pred, zero_division=0)
    f1  = f1_score(y_true, pred, zero_division=0)
    roc = roc_auc_score(y_true, proba)
    pr  = average_precision_score(y_true, proba)
    brier = brier_score_loss(y_true, proba)
    print(f"{name}: Acc={acc:.3f}, Prec={prec:.3f}, Rec={rec:.3f}, F1={f1:.3f}, ROC-AUC={roc:.3f}, PR-AUC={pr:.3f}, Brier={brier:.3f}")
    return {"accuracy":acc,"precision":prec,"recall":rec,"f1":f1,"roc_auc":roc,"pr_auc":pr,"brier":brier}

def print_confmat_and_report(y_true, proba, thr, tag):
    pred = (proba >= thr).astype(int)
    print(f"\nConfusion Matrix ({tag} @thr={thr:.3f}):\n", confusion_matrix(y_true, pred))
    print(f"\nClassification Report ({tag}):\n", classification_report(y_true, pred, digits=3))


In [ ]:
# Heart2020 tuned LightGBM (class_weight + our best-ish params)
pre_h = make_preprocessor(Xh_tr, scale_numeric=False)
lgb_h = LGBMClassifier(random_state=RANDOM_STATE, class_weight='balanced',
                       n_estimators=200, learning_rate=0.05, num_leaves=31,
                       max_depth=-1, min_child_samples=20, subsample=0.85,
                       colsample_bytree=0.85, reg_lambda=1.0)
pipe_h_lgb = Pipeline([("prep", pre_h), ("clf", lgb_h)])

# Cardio tuned LGBM
pre_c = make_preprocessor(Xc_tr, scale_numeric=False)
lgb_c = LGBMClassifier(random_state=RANDOM_STATE, class_weight='balanced',
                       n_estimators=200, learning_rate=0.05, num_leaves=31,
                       max_depth=-1, min_child_samples=20, subsample=0.85,
                       colsample_bytree=0.85, reg_lambda=1.0)
pipe_c_lgb = Pipeline([("prep", pre_c), ("clf", lgb_c)])

# UCI: tuned LGBM + tuned Logistic Regression
pre_u = make_preprocessor(Xu_tr, scale_numeric=True)
lgb_u = LGBMClassifier(random_state=RANDOM_STATE, class_weight='balanced',
                       n_estimators=200, learning_rate=0.03, num_leaves=31, max_depth=-1)
lr_u  = LogisticRegression(max_iter=5000, class_weight='balanced', C=0.1, solver="lbfgs")
pipe_u_lgb = Pipeline([("prep", pre_u), ("clf", lgb_u)])
pipe_u_lr  = Pipeline([("prep", pre_u), ("clf", lr_u)])


In [ ]:
from sklearn.ensemble import StackingClassifier

def fit_eval_stack(name, Xtr, ytr, Xte, yte, preprocess, use_xgb=False):
    base_lr  = LogisticRegression(max_iter=5000, class_weight='balanced', C=0.5, solver='lbfgs')
    base_lgb = LGBMClassifier(random_state=RANDOM_STATE, class_weight='balanced',
                              n_estimators=200, learning_rate=0.05, num_leaves=63,
                              max_depth=-1, min_child_samples=20, subsample=0.85,
                              colsample_bytree=0.85, reg_lambda=1.0)
    estimators = [("lr", base_lr), ("lgb", base_lgb)]
    if use_xgb:
        base_xgb = XGBClassifier(
            random_state=RANDOM_STATE, tree_method="hist", eval_metric="logloss",
            n_estimators=300, max_depth=5, learning_rate=0.07, subsample=0.8, colsample_bytree=0.8
        )
        estimators.append(("xgb", base_xgb))

    stack = Pipeline([
        ("prep", preprocess),
        ("stack",
         StackingClassifier(
             estimators=estimators,
             final_estimator=LogisticRegression(max_iter=2000, class_weight='balanced'),
             passthrough=True,
             cv=3,
             n_jobs=-1
         )
        )
    ])

    stack.fit(Xtr, ytr)
    proba = stack.predict_proba(Xte)[:,1]
    print(f"\n🧱 Stacking — {name}")
    m50 = report_test_metrics(f"{name} (stack @0.5)", yte, proba, 0.5)
    thr, f1b, pb, rb = best_f1_threshold(yte, proba)
    print(f"Best F1 threshold={thr:.3f} (F1={f1b:.3f}, Prec={pb:.3f}, Rec={rb:.3f})")
    mopt = report_test_metrics(f"{name} (stack @bestF1)", yte, proba, thr)
    print_confmat_and_report(yte, proba, thr, f"{name} stack")
    return stack, {"thr":thr, "m50":m50, "mopt":mopt}

# Heart2020 on a subsample for speed
h_frac = 0.40  # adjust if you want faster/slower
Xh_sub, _, yh_sub, _ = train_test_split(Xh_tr, yh_tr, train_size=h_frac, stratify=yh_tr, random_state=RANDOM_STATE)

stack_h, res_h = fit_eval_stack("Heart2020", Xh_sub, yh_sub, Xh_te, yh_te, pre_h, use_xgb=False)
stack_c, res_c = fit_eval_stack("Cardio",     Xc_tr, yc_tr, Xc_te, yc_te, pre_c, use_xgb=True)
stack_u, res_u = fit_eval_stack("UCI",        Xu_tr, yu_tr, Xu_te, yu_te, pre_u, use_xgb=True)

# Save stacks
joblib.dump(stack_h, f"{MODELS}/stack_heart2020.joblib")
joblib.dump(stack_c, f"{MODELS}/stack_cardio.joblib")
joblib.dump(stack_u, f"{MODELS}/stack_uci.joblib")
json.dump({"thr":res_h["thr"]}, open(f"{MODELS}/stack_heart2020_meta.json","w"))
json.dump({"thr":res_c["thr"]}, open(f"{MODELS}/stack_cardio_meta.json","w"))
json.dump({"thr":res_u["thr"]}, open(f"{MODELS}/stack_uci_meta.json","w"))



🧱 Stacking — Heart2020
Heart2020 (stack @0.5): Acc=0.742, Prec=0.219, Rec=0.787, F1=0.343, ROC-AUC=0.839, PR-AUC=0.350, Brier=0.170
Best F1 threshold=0.744 (F1=0.397, Prec=0.335, Rec=0.486)
Heart2020 (stack @bestF1): Acc=0.873, Prec=0.335, Rec=0.486, F1=0.397, ROC-AUC=0.839, PR-AUC=0.350, Brier=0.170

Confusion Matrix (Heart2020 stack @thr=0.744):
 [[53203  5281]
 [ 2814  2661]]

Classification Report (Heart2020 stack):
               precision    recall  f1-score   support

           0      0.950     0.910     0.929     58484
           1      0.335     0.486     0.397      5475

    accuracy                          0.873     63959
   macro avg      0.642     0.698     0.663     63959
weighted avg      0.897     0.873     0.884     63959


🧱 Stacking — Cardio
Cardio (stack @0.5): Acc=0.733, Prec=0.749, Rec=0.700, F1=0.724, ROC-AUC=0.799, PR-AUC=0.780, Brier=0.181
Best F1 threshold=0.349 (F1=0.742, Prec=0.671, Rec=0.830)
Cardio (stack @bestF1): Acc=0.712, Prec=0.671, Rec=0.830, F1=0

In [ ]:
def calibrate_and_compare(tag, base_pipe, Xtr, ytr, Xte, yte, save_prefix):
    # Fit base (if not yet)
    base_pipe.fit(Xtr, ytr)
    proba_base = base_pipe.predict_proba(Xte)[:,1]
    print(f"\n🔧 {tag} — Uncalibrated")
    m_base = report_test_metrics(f"{tag} (uncal)", yte, proba_base, 0.5)

    # Calibrated (sigmoid)
    cal = CalibratedClassifierCV(base_pipe, method="sigmoid", cv=3)
    cal.fit(Xtr, ytr)
    proba_cal = cal.predict_proba(Xte)[:,1]
    print(f"\n🎯 {tag} — Calibrated (sigmoid)")
    m_cal = report_test_metrics(f"{tag} (cal)", yte, proba_cal, 0.5)

    # Plot calibration curve
    pt_u, pp_u = calibration_curve(yte, proba_base, n_bins=10)
    pt_c, pp_c = calibration_curve(yte, proba_cal,  n_bins=10)

    plt.figure(figsize=(5,5))
    plt.plot(pp_u, pt_u, "s-", label="Uncalibrated")
    plt.plot(pp_c, pt_c, "o-", label="Calibrated")
    plt.plot([0,1],[0,1],"k--", alpha=0.7)
    plt.title(f"Calibration — {tag}")
    plt.xlabel("Predicted probability")
    plt.ylabel("True probability")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{OUT}/{save_prefix}_calibration.png", dpi=150)
    plt.close()

    return m_base, m_cal, cal

m_h_base, m_h_cal, cal_h = calibrate_and_compare("Heart2020 LGBM", pipe_h_lgb, Xh_tr, yh_tr, Xh_te, yh_te, "heart2020")
m_c_base, m_c_cal, cal_c = calibrate_and_compare("Cardio LGBM",    pipe_c_lgb, Xc_tr, yc_tr, Xc_te, yc_te, "cardio")
m_u_base, m_u_cal, cal_u = calibrate_and_compare("UCI LGBM",       pipe_u_lgb, Xu_tr, yu_tr, Xu_te, yu_te, "uci")

# Save calibrated models too
joblib.dump(cal_h, f"{MODELS}/cal_heart2020_lgbm.joblib")
joblib.dump(cal_c, f"{MODELS}/cal_cardio_lgbm.joblib")
joblib.dump(cal_u, f"{MODELS}/cal_uci_lgbm.joblib")


[LightGBM] [Info] Number of positive: 21898, number of negative: 233938
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.174463 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 432
[LightGBM] [Info] Number of data points in the train set: 255836, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000

🔧 Heart2020 LGBM — Uncalibrated
Heart2020 LGBM (uncal): Acc=0.732, Prec=0.215, Rec=0.806, F1=0.339, ROC-AUC=0.840, PR-AUC=0.351, Brier=0.168
[LightGBM] [Info] Number of positive: 14599, number of negative: 155958
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.060732 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 431
[LightGBM] [Info] Number of data

['/content/drive/MyDrive/heartriskx/models/day8/cal_uci_lgbm.joblib']

In [ ]:
def shap_dependence_topk(pipe, Xtr, tag, k=3):
    # Fit if needed, then transform training data
    pipe.fit(Xtr, np.zeros(len(Xtr))) if not hasattr(pipe, "predict_proba") else None
    X_enc = pipe.named_steps["prep"].fit_transform(Xtr)
    clf   = pipe.named_steps["clf"]
    feat_names = pipe.named_steps["prep"].get_feature_names_out()

    expl = shap.TreeExplainer(clf)
    sv = expl.shap_values(X_enc)
    # For binary LGBM, sv is a list [neg,pos]
    sv_pos = sv[1] if isinstance(sv, list) else sv
    mean_abs = np.abs(sv_pos).mean(axis=0)
    top_idx = np.argsort(mean_abs)[-k:][::-1]
    top_feats = [feat_names[i] for i in top_idx]

    # Save a dependence plot per top feature
    for f in top_feats:
        try:
            shap.dependence_plot(f, sv_pos, X_enc, feature_names=feat_names, show=False)
            plt.title(f"{tag} — SHAP dependence: {f}")
            safe = f.replace("/","_").replace("[","").replace("]","")
            plt.tight_layout()
            plt.savefig(f"{OUT}/{tag.lower()}_shap_dep_{safe}.png", dpi=140, bbox_inches="tight")
            plt.close()
        except Exception as e:
            print(f"SHAP dependence skipped for {f}: {e}")

# Use calibrated LGBM for nicer probabilities (cardio & uci), and plain for heart2020 due to size
shap_dependence_topk(pipe_h_lgb, Xh_tr.sample(min(5000, len(Xh_tr)), random_state=RANDOM_STATE), "Heart2020", k=3)
shap_dependence_topk(pipe_c_lgb, Xc_tr, "Cardio", k=3)
shap_dependence_topk(pipe_u_lgb, Xu_tr, "UCI", k=3)

print("✅ Saved SHAP dependence plots in:", OUT)


✅ Saved SHAP dependence plots in: /content/drive/MyDrive/heartriskx/outputs/day8


In [ ]:
def pdp_for_features(pipe, Xtr, Xte, feats, title_prefix, save_prefix):
    pipe.fit(Xtr, np.zeros(len(Xtr))) if not hasattr(pipe, "predict_proba") else None
    model = pipe
    # Only keep features present
    feats_avail = [f for f in feats if f in Xtr.columns]
    if not feats_avail:
        print(f"PDP skipped — no requested features present for {title_prefix}.")
        return
    try:
        disp = PartialDependenceDisplay.from_estimator(
            model, Xte, features=feats_avail, kind="average"
        )
        plt.suptitle(f"{title_prefix} — PDP")
        plt.tight_layout()
        plt.savefig(f"{OUT}/{save_prefix}_pdp.png", dpi=150, bbox_inches="tight")
        plt.close()
    except Exception as e:
        print(f"PDP failed for {title_prefix}: {e}")

# Choose typical numeric features:
# Heart2020 often has: BMI, PhysicalHealth, MentalHealth, SleepTime, AgeCategory (categorical)
pdp_for_features(pipe_h_lgb, Xh_tr, Xh_te, ["BMI","PhysicalHealth","SleepTime"], "Heart2020", "heart2020")

# Cardio: typical numeric columns
pdp_for_features(pipe_c_lgb, Xc_tr, Xc_te, ["ap_hi","ap_lo","age","weight"], "Cardio", "cardio")

# UCI: typical numeric columns
pdp_for_features(pipe_u_lgb, Xu_tr, Xu_te, ["age","chol","trestbps","thalach"], "UCI", "uci")

print("✅ Saved PDP plots to:", OUT)


✅ Saved PDP plots to: /content/drive/MyDrive/heartriskx/outputs/day8


In [ ]:
def quick_eval_table(name, base_pipe, stack_pipe, cal_model, Xte, yte):
    base_proba = base_pipe.predict_proba(Xte)[:,1]
    stack_proba= stack_pipe.predict_proba(Xte)[:,1]
    cal_proba  = cal_model.predict_proba(Xte)[:,1]

    entries = []
    for tag, proba in [("Base", base_proba), ("Stack", stack_proba), ("Calibrated", cal_proba)]:
        thr, f1b, pb, rb = best_f1_threshold(yte, proba)
        m = report_test_metrics(f"{name} {tag} (@bestF1)", yte, proba, thr)
        entries.append([tag, m["roc_auc"], m["pr_auc"], m["f1"], m["precision"], m["recall"], m["accuracy"], m["brier"], thr])
    cols = ["Model","ROC-AUC","PR-AUC","F1","Precision","Recall","Accuracy","Brier","BestThr"]
    df = pd.DataFrame(entries, columns=cols)
    print(f"\n{name} — Summary:\n", df.sort_values("PR-AUC", ascending=False))
    return df

tbl_h = quick_eval_table("Heart2020", pipe_h_lgb, stack_h, cal_h, Xh_te, yh_te)
tbl_c = quick_eval_table("Cardio",    pipe_c_lgb, stack_c, cal_c, Xc_te, yc_te)
tbl_u = quick_eval_table("UCI",       pipe_u_lgb, stack_u, cal_u, Xu_te, yu_te)

tbl_h.to_csv(f"{OUT}/summary_heart2020.csv", index=False)
tbl_c.to_csv(f"{OUT}/summary_cardio.csv", index=False)
tbl_u.to_csv(f"{OUT}/summary_uci.csv", index=False)
print("📄 Saved comparison CSVs in:", OUT)


Heart2020 Base (@bestF1): Acc=0.876, Prec=0.339, Rec=0.479, F1=0.397, ROC-AUC=0.840, PR-AUC=0.351, Brier=0.168
Heart2020 Stack (@bestF1): Acc=0.873, Prec=0.335, Rec=0.486, F1=0.397, ROC-AUC=0.839, PR-AUC=0.350, Brier=0.170
Heart2020 Calibrated (@bestF1): Acc=0.871, Prec=0.332, Rec=0.499, F1=0.398, ROC-AUC=0.840, PR-AUC=0.352, Brier=0.066

Heart2020 — Summary:
         Model   ROC-AUC    PR-AUC        F1  Precision    Recall  Accuracy  \
2  Calibrated  0.840387  0.352065  0.398395   0.331633  0.498813  0.871042   
0        Base  0.840361  0.350598  0.397366   0.339279  0.479452  0.875514   
1       Stack  0.838934  0.350443  0.396661   0.335054  0.486027  0.873435   

      Brier   BestThr  
2  0.065840  0.232747  
0  0.167623  0.743948  
1  0.169549  0.744202  
Cardio Base (@bestF1): Acc=0.707, Prec=0.663, Rec=0.842, F1=0.742, ROC-AUC=0.800, PR-AUC=0.784, Brier=0.181
Cardio Stack (@bestF1): Acc=0.712, Prec=0.671, Rec=0.830, F1=0.742, ROC-AUC=0.799, PR-AUC=0.780, Brier=0.181
Cardio Cali